<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src="http://opencloud.utsa.edu/wp-content/themes/utsa-oci/images/logo.png"/>

### **University of Texas at San Antonio** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.5em;"> **Open Cloud Institute** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### OpenStack Cloud Networking

<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Paul Rad, Ph.D.** </span>  


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Gonzalo De La Torre, Ph.D. Student** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> *Open Cloud Institute, University of Texas at San Antonio, San Antonio, Texas, USA* </span>  
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> {gonzalo.delatorreparra, paul.rad}@utsa.edu </span>  

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

## NEUTRON - OpenStack Networking Services

### Introduction

#### Definition

Neutron is an OpenStack project to provide "networking as a service" between interface devices (e.g., vNICs) managed by other Openstack services (e.g., nova).

#### Why Neutron

 It is increasingly becoming difficult for data centers to provide network to a platform whose network requirements keep     changing from time to time. Old traditional ways for delivering network topologies and infrastructure is no longer the norm
 of the day. Clients want instantaneous network provision on demand. To cope up with this, Openstack has launched the neutron
 service. From openstack’s perspective, main goals of neutron are: 

- Providing connectivity to virtual machines with each other and external network.
- Maintaining the isolation in communication of VMs belonging to different tenants. Keep   each tenant isolated.
- Help tenants create their own Networks, routers, complex networking topologies etc. Idea of networking virtualization.
- *aas as a service. Ex: Load Balancer as a service, Firewall as a service.


#### Core Concepts of Neutron

 Neutron utilizes three main resources:

 - Ports: A connection point for attaching a single device NIC to neutron network.
 - Network: An isolated L2 segment analogous to VLAN. Also called as a broadcast domain.
 - Subnets: A range of V4 or V6 IP addresses associated with a configuration state.
 ![Image](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/neutron/images/coreconcepts.png?raw=True)

### Architecture

There are many aspects of neutrons architecture that we need to discuss. First we will start discussing the various components of the neutron architecture. Then we shall dig deep into each component. 

  ![Image2](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/neutron/images/blockdiagram.png?raw=True)

#### The Block diagram
 

The above block shows the core components of neutron. Neutron Server component receives all the API calls and translates them to respective plugins for implementing the desired functionality. For traction, management and analytics, each service rendered by neutron is persisted in the neutron database. This database is a typical Maria DB installation and can be installed in the same controller node as the neutron server or can be installed on a separate node. Plugins give the neutron the capability to provide varied network services. Being pluggable with various plugins has given neutron the required potential to deliver any complicated network topologies that a tenant requires and also provides the necessary capabilities to modify these networks with least downtime. It is not always necessary that all the restful API requests received by neutron server are handled by plugins. 

The original OpenStack Compute network implementation assumed a very basic model of performing all isolation through Linux VLANs and IP tables. OpenStack Networking introduces the concept of a plug-in, which is a pluggable back-end implementation of the OpenStack Networking API. A plug-in can use a variety of technologies to implement the logical API requests. Some OpenStack Networking plug-ins might use basic Linux VLANs and IP tables, while others might use more advanced technologies, such as L2-in-L3 tunneling or OpenFlow, to provide similar benefits.

Plugin agent (quantum-*-agent): Runs on each hypervisor to perform local vswitch configuration (if your neutron configuration is based on Open VSwitch technology). Agent to be run depends on which plug-in you are using, as some plug-ins do not require an agent. DHCP agent (quantum-dhcp-agent): Provides DHCP services to tenant networks. This agent is the same across all plug-ins. L3 agent (quantum-L3-agent): Provides L3/NAT forwarding to provide external network access for VMs on tenant networks. This agent is the same across all plug-ins.

Plug-ins and various internal components of neutron utilize messaging queue to communicate with each other. Most of these communications are RPC (remote procedure call) requests for fulfilling API request. 


#### Types of networks in Openstack

 
![Image3](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/neutron/images/typesofnetworks.png?raw=True)

**Provider Network**: Created by admin, mapped to pre-existing network in datacenter, used for external networks. VMs may also be directly connected to provider network. Provider network are mainly used to model or simulate all or few of the physical networking hardware.

**Tenant Network**: These are the networks created by neutron for the tenants on a request sent by an authorized user of that tenant. These networks are isolated from each other. These networks act as virtual private networks belonging to individual tenants and each VM belonging to that tenant is connected to that tenant network.

 Such tenant networks cannot communicate to external network, or receive communication from external network unless they somehow hook-up with one of the provider networks provided by the network administrator of that data center.


#### Communications

As we all might have realized by now that all the components of neutron are necessarily installed on a single node. And also these components keep communicating with each other to get the job done. Communication between various Openstack components and services can be classified broadly into four types. 

 ![Image4](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/neutron/images/2.3%20communication.png?raw=True)

**Management Communication**: This kind of communication takes place between various components of Openstack services in order to work together to accomplish the requested task. Most of the components may use RPC means of communication using RabbitMQ. This sort of communication uses separate network other than the Tenant or provider network we discussed earlier.

**VM Data**: This comprises of the communication of VMs within a tenant network. 

**API Network**: This network enables calling the API server requesting for implementation of various services.
Internet: Acceptable external network.

 
![Image4](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/neutron/images/Acceptable%20external%20network.png?raw=True{width=1000px height=1000px})

### Creating your Own Network

We will start our hands-on lesson by requesting OpenStack the current list of available networks:

In [1]:
%%bash
. /home/stack/devstack/openrc
neutron net-list

+--------------------------------------+---------+----------------------------------------------------------+
| id                                   | name    | subnets                                                  |
+--------------------------------------+---------+----------------------------------------------------------+
| 3e107a98-4b4a-4fd8-8bb3-d3ef1b4bf7a4 | public  | 514aacc6-23d0-482d-ab95-0c112e62ad6f                     |
|                                      |         | 26175483-4bbb-4fc2-9408-d90f878da86b                     |
| c28dd3c2-d5f3-4d72-8d36-e8aac25b185c | private | 55bb7d26-654d-4ec4-a259-37a3a79db8f7 fd11:2bc4:625b::/64 |
|                                      |         | d8e42c8c-b55b-4afb-a695-6c2de8aed647 192.168.1.0/24      |
+--------------------------------------+---------+----------------------------------------------------------+


### Creating a Public Network

The first step that we need to follow whenever we start creating our first functional network topology is the creation of our own Public Network. The public network needs to be created based on the physical network where your OpenStack environment is deployed.

In [2]:
%%bash
. /home/stack/devstack/openrc
neutron net-create --shared --provider:network_type flat --provider:physical_network flat --router:external <Name of Network>

bash: line 2: syntax error near unexpected token `newline'
bash: line 2: `neutron net-create --shared --provider:network_type flat --provider:physical_network flat --router:external <Name of Network>'


In [4]:
%%bash
. /home/stack/devstack/openrc
neutron subnet-create --name <subnet name> --gateway 17.200.0.1 --allocation-pool start=17.200.50.2,end=17.200.50.255 --dns-nameserver 8.8.8.8 <Name of your public network> 17.200.0.0/16

SyntaxError: invalid syntax (<ipython-input-4-a3dd19150211>, line 1)

### Creating a Private Network

Private network creation can be performed with user or admin credentials. This depends on the permissions defined in the policy.json file in Neutron. Let us proceed to create a private network by using the command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
neutron net-create my-private-network

In the same manner as we experienced in the public network, the above command will only create a database entry.
To specify the details of our private network, we need to create a subnet for this private network. The specifics of this private subnet do not need to be based on the physical network as this network will only be used by neutron as an internal network. To create a subnet, we can simply use the command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
neutron subnet-create my-private-network --name my-private-subnet --gateway 192.168.0.1 192.168.0.0/24

### Creating a Router

Now that we have created our Public and Private networks, we need to proceed to create a router that will enable communication between both of them. To create our router we can use the command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
neutron router-create my-router

### Adding Interfaces Between Router and Subnets

Note that this will only create the router but so far there is no connectivity between this and any of the previously created networks. To create the connectivity we need, we require to add interfaces from the router to our networks. We can proceed to create these interfaces by using the commands:

In [ ]:
%%bash
. /home/stack/devstack/openrc
neutron router-interface-add my-router my-public-subnet

This will create an interface that will be attached to the public subnet.

In [ ]:
%%bash
. /home/stack/devstack/openrc
neutron router-interface-add my-router my-private-subnet

This will create an interface that will be attached to the private subnet. Next, we will look in to the Nova service to create an instance or VM.

## Nova - OpenStack Compute Service

## 1. Introdunction

Nova is all about access to compute resources.  

Nova is used for hosting and managing cloud computing systems.  It is a component based architecture enabling quicker additions of new features.  It is scalable, on-demand, fault tolerant, recoverable and provides API-compatibility with systems like Amazon EC2.  Nova provides compute resources such as virtual machines, containers, and bare metal servers.

Nova is built on a messaging architecture.  All of its components can typically be run on several servers.  This architecture allows the components to communicate through a message queue. Deferred objects are used to avoid blocking while a component waits in the message queue for a response.

As the most distributed component in the OpenStack platform, Nova interacts heavily with other OpenStack services like Keystone for performing authentication, Horizon for its Web interface and Glance for supplying its images.

### 2. Architecture

![Architecture Diagram](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/nova/resources/Architecture/Nova_Architecture.png?raw=True)

##### 2.1 Nova REST API
This is the Nova component that receives HTTP requests, converts commands and communicates with other components via the oslo.messaging queue or HTTP.

<ul>
<li>list servers (GET)
<li>Create server (POST)
<li>List details for servers (GET)
<li>Show server details (GET)
<li>Update server (PUT)
<li>Delete server (DELETE)</ul>

##### 2.2 Messaging
Advanced Message Queueing Protocol (AMQP) is a messaging protocol used by OpenStack allowing Nova components to communicate with each other. AMQP uses a broker, either RabbitMQ or Qpid, to allow these Nova components to send Remote Procedure Calls (RPC) using a publish/subscribe paradigm.

The architecture is shown in the following picture:</br> 

![AMQP Architecture Image](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/nova/resources/Architecture/AMQP_Architecture_Image.png?raw=True)

##### 2.3 Scheduler

The Filter Scheduler uses filtering and weighting to make informed decisions on where a new instance should be created on a Compute Node.

![Threading Model](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/nova/resources/Architecture/Threading_Model.png?raw=True)

The Filter Scheduler looks over all compute nodes and evaluates them against a set of filters. The filters will eliminate some of the hosts, and the resulting hosts will be weighted, which will sort them by suitability. The Scheduler will then choose the hosts for each instance based on the weights. It is possible that the Scheduler may not find any candidate for the next instance, in which case that instance will not be booted. If the default scheduling algorithm is insufficient for a users needs, that user can create their own scheduling algorithm. There are a lot of built in functions that can be used to define the filtering algorithm.

The weighing process is defined by equations which can also be set by the user. Different properties, such as RAM usage, CPU usage, Disk usage, I/O usage, etc, can be assigned different values in order to use a custom sorting to choose which host is the most suitable for the new instance(s).

![Weighing](https://github.com/cloudandbigdatalab/OpenStack-Projects/blob/master/nova/resources/Architecture/Weighing.png?raw=True)


##### 2.4 Conductor
In addition to its roles as a database proxy and object backporter the conductor service also serves as a centralized place to manage the execution of workflows which involve the scheduler. Rebuild, resize/migrate, and building an instance are managed here. This was done in order to have a better separation of responsibilities between what compute nodes should handle and what the scheduler should handle, and to clean up the path of execution. Conductor was chosen because in order to query the scheduler in a synchronous manner it needed to happen after the API had returned a response otherwise API response times would increase. And changing the scheduler call from asynchronous to synchronous helped to clean up the code.

##### 2.5 Compute
Compute manages communication with hypervisors and virtual machines. You can create compute nodes that will receive requests from the controller node and virtual machine instances. The compute service relies on a hypervisor to run virtual machine instances. OpenStack can use a number of various hypervisors such as KVM, QEMU, Xen, or LXC.

The basics: Compute will accept actions from the queue and then perform a series of system commands and carry them out while updating status in the database.

While all services are designed to be horizontally scalable, you should have significantly more computes then anything else.

### 3. Excercise

Now let’s do some hands-on using Nova CLI.

To know about sub-commands and supported options by Nova use the below command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova help

First for booting the instance you need to select the desired image to get started. For listing different image types use the below command: 

In [6]:
%%bash
. /home/stack/devstack/openrc
glance image-list

+--------------------------------------+--------------------------------------------------+-------------+------------------+-------------+--------+
| ID                                   | Name                                             | Disk Format | Container Format | Size        | Status |
+--------------------------------------+--------------------------------------------------+-------------+------------------+-------------+--------+
| 2808a72c-df39-41d9-b03a-16d1eab8a79a | arun-tensorflow-notebook                         | qcow2       | bare             | 5308481536  | active |
| a9228c93-f8b9-4067-820f-5f04d9ca4776 | Caffe_Complete                                   | qcow2       | bare             | 6613696512  | active |
| be46bd5a-c4a5-4495-ad30-356186d8ff04 | CC-C7-autologin                                  | qcow2       | bare             | 342360064   | active |
| 8426d712-9742-4eff-81cf-d886a4ba2cdd | CC-CENTOS-3656275b-824c-4e3b-bdb1-fe7a61e77eaf   | qcow2       | bare  

Next after deciding on the image to boot run the below command to start the instance: 

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova boot --flavor m1.tiny --image cirros-0.3.4-x86_64-uec my-instance my_instance_name

As you can see that we have specified some of the parameters like --flavor and --image for our instance. To get list of flavors supported run the below command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova flavor-list

For knowing the list of instances running in the environment run the below command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova list

At any point of time, to know the details of the instance that you created use the below command: 

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova show <my_instance_id>

To pause an instance, run the following command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova pause <name-of-the-instance>

This command stores the state of the VM in RAM. Also we can see that the instance has been paused in power state tab.
A paused instance continues to run in a frozen state.To unpause an instance, run the following command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova unpause <name-of-the-instance>

To stop or start an instance you can use the command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova suspend <name-of-the-instance>

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova resume <name-of-the-instance>

To see the list of all the nova services, you can use the command:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova service-list

Now we can login in to the created VM:

In [ ]:
ip netns exec qdhcp-<network UUID> ssh cirros@<ip address>

The netork UUID information can be found by executing the command **ip netns list** and ip address of the host can be found from **nova list** command. Note, in CirrOS the password for user cirros is "cubswin:)" without the quotes.

Network namespaces will only be seen on nodes running the Neutron DHCP,
L3, and LBaaS agents. These services are typically configured only on controllers
or dedicated network nodes.

If at any point something goes wrong or you don’t want the created instance any more you can always delete with:

In [ ]:
%%bash
. /home/stack/devstack/openrc
nova delete <name-of-instance>